Большая часть кода ниже опиралась на класс, изменявшийся в процессе работы, так что не все клетки воспроизводятся. Будет явно указано, какие результаты возможно воспроизвести.

In [1]:
from tg.grammar_ru.ml.corpus import CorpusReader
from tg.grammar_ru.common import Loc
from pathlib import Path

base_path = Loc.corpus_path/'example.base.zip'
reader = CorpusReader(Path(base_path))

In [2]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


Конечно, никто никогда в мире не писал замену имён собственных, поэтому напишем:

In [3]:
alg.pronoun_replacer

{'академия': 'женщина',
 'алевтина': 'женщина',
 'амелия': 'женщина',
 'андрий': 'мужчина',
 'аннегрета': 'женщина',
 'бэнд': 'мужчина',
 'ванко': 'мужчина',
 'верин': 'мужчина',
 'викторович': 'женщина',
 'глас': 'мужчина',
 'густав': 'мужчина',
 'денежкин': 'мужчина',
 'исповедание': 'существо',
 'кареолан': 'мужчина',
 'карл': 'мужчина',
 'каролина': 'женщина',
 'каспер': 'мужчина',
 'кик': 'мужчина',
 'кит': 'мужчина',
 'лилиана': 'женщина',
 'лилия': 'женщина',
 'марек': 'мужчина',
 'мартин': 'мужчина',
 'матвей': 'мужчина',
 'михал': 'мужчина',
 'никита': 'мужчина',
 'оливер': 'мужчина',
 'ополе': 'существо',
 'патрик': 'мужчина',
 'росинтеллектсистема': 'женщина',
 'снегур': 'мужчина',
 'стефанос': 'мужчина',
 'творец': 'мужчина',
 'тор': 'мужчина',
 'торам': 'мужчина',
 'франтишек': 'мужчина',
 'фредерик': 'мужчина',
 'ханна': 'женщина',
 'ханне': 'женщина',
 'чисовица': 'женщина',
 'ярина': 'женщина',
 'ярином': 'мужчина'}

In [2]:
db = reader.get_bundles().first()               

In [5]:
# то, что, вероятно, было выделением имён собственных в slovnet:
from tg.grammar_ru.ml.features import SlovnetFeaturizer

featurizer = SlovnetFeaturizer()
featurizer.featurize(db)
db.src[db.slovnet.POS == 'PROPN'].word.head(15)

0              Лилия
63          Закричав
65             Лилия
118           Голова
142            Лилия
152         Выманили
208            Лилия
220    Прикоснувшись
275            Лилия
333        Чисовицах
346        Девятерых
355           Лилией
370         Академии
372         Каролина
402            Лилия
Name: word, dtype: object

In [6]:
from tg.grammar_ru.ml.features import PyMorphyFeaturizer

featurizer = PyMorphyFeaturizer()
featurizer.featurize(db)
db.pymorphy['normal_form'] = db.pymorphy['normal_form'].apply(lambda x: alg.pronoun_replacer[x] if x in alg.pronoun_replacer else x)
db.pymorphy.head()

,normal_form,alternatives,score,delta_score,POS,animacy,gender,number,case,aspect,transitivity,person,tense,mood,voice,involvement
word_id,,,,,,,,,,,,,,,,
0,женщина,2,0.666666,0.333333,NOUN,anim,femn,sing,nomn,NaN,NaN,NaN,NaN,NaN,NaN,None
1,очнуться,1,1.000000,1.000000,VERB,NaN,femn,sing,NaN,perf,intr,NaN,past,indc,NaN,None
2,от,1,1.000000,1.000000,PREP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,суетливый,3,0.333333,0.000000,ADJF,NaN,masc,sing,gent,NaN,NaN,NaN,NaN,NaN,NaN,None
4,",",1,1.000000,1.000000,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [7]:
candidates_df = alg.get_candidates(db)
with_parents_df = alg.get_pronoun_parent(db, candidates_df)
with_parents_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id
0,10,0,0,9
1,18,0,1,19
2,18,10,0,19
3,30,0,2,31
4,30,10,1,31
...,...,...,...,...
303,921,907,4,922
304,921,914,3,922
305,921,917,2,922
306,921,918,1,922


In [8]:
from tg.grammar_ru.ml.features import NavecFeaturizer

product_df = with_parents_df.merge(db.pymorphy.normal_form.rename('pronoun_parent_norm'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = product_df.merge(db.pymorphy.normal_form.rename('candidate_word_norm'), left_on='candidate_word_id', right_index=True, how='left')
featurizer = NavecFeaturizer()
product_df['product'] = featurizer.get_glove_prod(product_df, 'pronoun_parent_norm', 'candidate_word_norm')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,9,преследовать,женщина,2.660135
1,18,0,1,19,гнаться,женщина,-1.817963
2,18,10,0,19,гнаться,она,-0.213960
3,30,0,2,31,оставаться,женщина,7.911864
4,30,10,1,31,оставаться,она,8.539292
...,...,...,...,...,...,...,...
303,921,907,4,922,кубарем,жертва,-0.980053
304,921,914,3,922,кубарем,судьба,-1.951545
305,921,917,2,922,кубарем,она,-0.423255
306,921,918,1,922,кубарем,роль,-1.972080


In [9]:
# Можно ли улучшить выбор кандидатов, если отбрасывать детей родителя местоимения:
pronoun_parent_ids = db.slovnet[db.slovnet.index.isin(candidates_df['pronoun_word_id'])]['syntax_parent_id']
candidate_parent_ids = db.slovnet[db.slovnet.index.isin(candidates_df['candidate_word_id'])]['syntax_parent_id']
parents_compare = (candidates_df
 .merge(pronoun_parent_ids.to_frame().reset_index(), left_on='pronoun_word_id', right_on='word_id')
 .merge(candidate_parent_ids.to_frame().reset_index(), left_on='candidate_word_id', right_on='word_id'))
parents_compare[parents_compare['syntax_parent_id_x'] == parents_compare['syntax_parent_id_y']]

,pronoun_word_id,candidate_word_id,candidate_distance,word_id_x,syntax_parent_id_x,word_id_y,syntax_parent_id_y
303,917,914,0,917,916,914,916


In [10]:
product_df[product_df['pronoun_word_id'] == 917]

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
294,917,878,6,916,уготовить,паника,-2.770621
295,917,884,5,916,уготовить,женщина,-5.830828
296,917,894,4,916,уготовить,лестница,-2.799802
297,917,896,3,916,уготовить,женщина,-5.830828
298,917,905,2,916,уготовить,она,-5.512835
299,917,907,1,916,уготовить,жертва,-0.058948
300,917,914,0,916,уготовить,судьба,3.859842


На первый DataBundle всего один случай ошибки при совпадающем у местоимения и кандидата в антецеденты родителе, осталось бы ошибкой, вероятно, пока можно не трогать.
UPD: всё-таки добавила такой фильтр.

In [11]:
def get_big_deviation_only(df):
    product_groups = df.drop_duplicates(['pronoun_parent_norm','candidate_word_norm'], keep='last').groupby(['pronoun_word_id'])['product']
    maxes = product_groups.max()
    means = product_groups.mean()
    stds = product_groups.std()
    max_indices = product_groups.idxmax().dropna().astype(int)
    best_in_group = df.loc[max_indices]
    temp = ((product_groups.count() < 4) | (stds < maxes - means)).to_frame()
    return best_in_group[best_in_group.pronoun_word_id.isin(temp[temp['product']].index)]

get_big_deviation_only(product_df)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,9,преследовать,женщина,2.660135
2,18,10,0,19,гнаться,она,-0.213960
5,30,18,0,31,оставаться,она,8.539292
15,53,30,0,52,подвести,она,1.963640
20,56,53,0,57,упасть,она,7.820927
23,83,56,4,84,увидеть,она,10.808409
30,113,95,4,114,напугать,девушка,5.793210
40,122,113,1,123,чувствовать,она,10.140821
42,145,105,6,146,подкараулить,кирпичный,-0.722384
53,160,124,2,163,успеть,тревога,5.222582


Брать местоимения в кандидаты было плохой идеей, далее без них.

In [2]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


In [4]:
alg.run_full(db).drop(['pronoun_parent_id'], axis=1)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,преследовать,женщина,2.660135
1,18,0,0,гнаться,женщина,-1.817963
2,30,0,0,оставаться,женщина,7.911864
9,53,0,0,подвести,женщина,-0.543144
10,56,0,0,упасть,женщина,5.786184
13,83,65,2,увидеть,женщина,8.552348
18,113,95,4,напугать,девушка,5.793210
31,145,104,5,подкараулить,глухой,-0.551586
42,160,124,1,успеть,тревога,5.222582
50,174,142,0,показаться,женщина,7.261024


In [76]:
from tg.grammar_ru.ml.features import NavecFeaturizer
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def inflect(word, case):
    return morph.parse(word)[0].inflect({'sing', case}).word

def get_inflected_candidates(db, with_parents_df):
    inflect_base = with_parents_df.merge(db.pymorphy, left_on='pronoun_word_id', right_on='word_id')[['candidate_word_id', 
                                                                                                 'case']]
    inflect_base = inflect_base.merge(db.pymorphy.reset_index()[['word_id', 'normal_form']], left_on='candidate_word_id', right_on='word_id', how='left')[['case', 'normal_form']]
    return inflect_base.apply(lambda x: inflect(x['normal_form'], x['case']), axis=1)

candidates_df = alg.get_candidates(db)
with_parents_df = alg.get_pronoun_parent(db, candidates_df)
product_df = with_parents_df.merge(db.src.word.str.lower().rename('pronoun_parent'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df['inflected_candidate'] = get_inflected_candidates(db, with_parents_df)
featurizer = NavecFeaturizer()
product_df['product'] = featurizer.get_glove_prod(product_df, 'pronoun_parent', 'inflected_candidate')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent,inflected_candidate,product
0,10,0,0,9,преследовал,женщину,8.436619
1,18,0,0,19,гналась,женщина,1.514307
2,30,0,0,31,оставался,женщине,3.133473
3,48,3,5,46,оторваться,суетливого,-0.075657
4,48,5,4,46,оторваться,путаного,-2.126049
...,...,...,...,...,...,...,...
291,921,896,4,922,кубарем,женщина,-0.210612
292,921,907,3,922,кубарем,жертва,-0.980053
293,921,914,2,922,кубарем,судьба,-1.951545
294,921,918,1,922,кубарем,роль,-1.972080


,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent,inflected_candidate,product
0,10,0,0,9,преследовал,женщину,8.436619
1,18,0,0,19,гналась,женщина,1.514307
2,30,0,0,31,оставался,женщине,3.133473
5,48,6,3,46,оторваться,сна,8.170823
9,53,0,0,52,подвели,женщину,5.742084
10,56,0,0,57,упала,женщина,12.184425
13,83,65,2,84,увидела,женщина,15.119053
18,113,95,4,114,напугала,девушка,5.132144
24,122,95,5,123,чувствовала,девушка,15.705741
32,145,105,4,146,подкараулили,кирпичного,4.122867


Стало ли лучше? Да, немного, стоит склонять. Кандидаты с расстоянием 5+ часто неправильные.
Кроме того оставлю в кандидатах только 'NOUN' и 'PROPN'.

In [3]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


In [13]:
alg.run_full(db)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent,inflected_candidate,product
0,10,0,0,9,преследовал,женщину,8.436619
1,18,0,0,19,гналась,женщина,1.514307
2,30,0,0,31,оставался,женщине,3.133473
4,48,6,2,46,оторваться,сна,8.170823
7,53,0,0,52,подвели,женщину,5.742084
8,56,0,0,57,упала,женщина,12.184425
11,83,65,1,84,увидела,женщина,15.119053
18,113,95,1,114,напугала,девушка,5.132144
24,122,95,2,123,чувствовала,девушка,15.705741
33,145,142,0,146,подкараулили,женщину,1.016804


Ок, проверка произведения для пары (кандидат, родитель местоимения) всё-таки даёт больше ошибок чем хотелось бы. Что если смотреть произведение (кандидат, сосед местоимения)?

In [6]:
candidates_df = alg.get_candidates(db)
with_right = candidates_df.copy()
with_right['right_id'] = candidates_df['pronoun_word_id'].apply(lambda x: x + 1 if db.src.loc[x + 1].word_type != 'punct' else None)
with_right = with_right.dropna().astype(int)
with_right

,pronoun_word_id,candidate_word_id,candidate_distance,right_id
1,18,0,0,19
2,30,0,0,31
8,56,0,0,57
9,83,0,3,84
10,83,61,2,84
...,...,...,...,...
288,921,896,4,922
289,921,907,3,922
290,921,914,2,922
291,921,918,1,922


In [41]:
from tg.grammar_ru.ml.features import NavecFeaturizer
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def inflect(word, case):
    return morph.parse(word)[0].inflect({'sing', case}).word

def get_inflected_candidates(db, with_parents_df):
    inflect_base = with_parents_df.merge(db.pymorphy, left_on='pronoun_word_id', right_on='word_id')[['candidate_word_id', 
                                                                                                 'case']]
    inflect_base = inflect_base.merge(db.pymorphy.reset_index()[['word_id', 'normal_form']], left_on='candidate_word_id', right_on='word_id', how='left')[['case', 'normal_form']]
    return inflect_base.apply(lambda x: inflect(x['normal_form'], x['case']), axis=1)

prod_df = with_right.merge(db.src.word.str.lower().rename('right'), left_on='right_id', right_index=True, how='left').reset_index(drop=True)
prod_df['inflected_candidate'] = get_inflected_candidates(db, with_right)
featurizer = NavecFeaturizer()
prod_df['product'] = featurizer.get_glove_prod(prod_df, 'right', 'inflected_candidate')

def get_big_deviation_only(df):
    product_groups = df.drop_duplicates(['right','inflected_candidate'], keep='last').groupby(['pronoun_word_id'])['product']
    maxes = product_groups.max()
    means = product_groups.mean()
    stds = product_groups.std()
    max_indices = product_groups.idxmax().dropna().astype(int)
    best_in_group = df.loc[max_indices]
    temp = ((product_groups.count() < 3) | (stds < maxes - means)).to_frame()
    return best_in_group[best_in_group.pronoun_word_id.isin(temp[temp['product']].index)]

get_big_deviation_only(prod_df)

,pronoun_word_id,candidate_word_id,candidate_distance,right_id,right,inflected_candidate,product
0,18,0,0,19,гналась,женщина,1.514307
1,30,0,0,31,оставался,женщине,3.133473
2,56,0,0,57,упала,женщина,12.184425
5,83,65,1,84,увидела,женщина,15.119053
12,113,95,1,114,напугала,девушка,5.132144
18,122,95,2,123,чувствовала,девушка,15.705741
27,145,142,0,146,подкараулили,женщину,1.016804
30,160,95,4,161,даже,девушка,9.747227
37,174,95,4,175,показалось,девушке,11.728028
49,224,95,6,225,мысленно,девушка,10.007082


Просто интересно попробовать: посчитать произведения для пар (кандидат, родитель), (кандидат, сосед слева), (кандидат, сосед справа)

In [4]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

In [13]:
from tg.grammar_ru.ml.features import NavecFeaturizer
import numpy as np
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def inflect(word, case):
    return morph.parse(word)[0].inflect({'sing', case}).word

def get_inflected_candidates(db, with_parents_df):
    inflect_base = with_parents_df.merge(db.pymorphy, left_on='pronoun_word_id', right_on='word_id')[['candidate_word_id', 
                                                                                                 'case']]
    inflect_base = inflect_base.merge(db.pymorphy.reset_index()[['word_id', 'normal_form']], left_on='candidate_word_id', right_on='word_id', how='left')[['case', 'normal_form']]
    return inflect_base.apply(lambda x: inflect(x['normal_form'], x['case']), axis=1)

def get_glove_prod_e_norm(df, col1, col2):
    copy = df.copy()
    featurizer = NavecFeaturizer()
    copy['prod_e'] = featurizer.get_glove_prod(copy, col1, col2).apply(lambda x: np.e ** x)
    return copy.groupby('pronoun_word_id')['prod_e'].transform(lambda x: x / x.sum()).fillna(0)

def get_extended_df(db, with_parents_df):
    extended_df = with_parents_df.copy()
    extended_df['right_id'] = candidates_df['pronoun_word_id'].apply(lambda x: x + 1 if db.src.loc[x + 1].word_type != 'punct' else -1)
    extended_df['left_id'] = candidates_df['pronoun_word_id'].apply(lambda x: x - 1 if (x > 0 and db.src.loc[x - 1].word_type != 'punct') else -1)

    extended_df['inflected_candidate'] = get_inflected_candidates(db, extended_df)
    extended_df = extended_df.drop_duplicates(['inflected_candidate', 'pronoun_word_id'],keep='last').reset_index(drop=True)
    extended_df = extended_df.merge(db.src.word.str.lower().rename('right_word'), left_on='right_id', right_index=True, how='left').reset_index(drop=True)
    extended_df = extended_df.merge(db.src.word.str.lower().rename('left_word'), left_on='left_id', right_index=True, how='left').reset_index(drop=True)
    extended_df = extended_df.merge(db.src.word.str.lower().rename('parent_word'), left_on='pronoun_parent_id', right_index=True, how='left')
    
    extended_df['left_prod_e'] = get_glove_prod_e_norm(extended_df, 'inflected_candidate', 'left_word')
    extended_df['right_prod_e'] = get_glove_prod_e_norm(extended_df, 'inflected_candidate', 'right_word')
    extended_df['parent_prod_e'] = get_glove_prod_e_norm(extended_df, 'inflected_candidate', 'parent_word')
    extended_df['sum'] = extended_df[['left_prod_e', 'right_prod_e', 'parent_prod_e']].sum(axis=1)
    return extended_df

In [43]:
import pandas as pd
import numpy as np

db = reader.get_bundles().first()
candidates_df = alg.get_candidates(db)
db.pymorphy['normal_form'] = db.pymorphy['normal_form'].apply(lambda x: alg.pronoun_replacer[x] if x in alg.pronoun_replacer else x)

extended_df = get_extended_df(db, alg.get_pronoun_parent(db, candidates_df))
extended_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,1.000000,0.000000,1.000000,2.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,1.000000,1.000000,2.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,1.000000,1.000000,2.000000
3,48,3,3,46,-1,47,суетливого,NaN,от,оторваться,0.001035,0.000000,0.000233,0.001267
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,0.992254,0.000000,0.887464,1.879718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,921,896,4,922,922,-1,женщина,кубарем,NaN,кубарем,0.000000,0.000492,0.000492,0.000984
240,921,907,3,922,922,-1,жертва,кубарем,NaN,кубарем,0.000000,0.000228,0.000228,0.000456
241,921,914,2,922,922,-1,судьба,кубарем,NaN,кубарем,0.000000,0.000086,0.000086,0.000173
242,921,918,1,922,922,-1,роль,кубарем,NaN,кубарем,0.000000,0.000084,0.000084,0.000169


In [42]:
max_indices = extended_df.groupby('pronoun_word_id')['sum'].idxmax().dropna().astype(int)
best_in_group = extended_df.loc[max_indices]
best_in_group

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,1.000000,0.000000,1.000000,2.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,1.000000,1.000000,2.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,1.000000,1.000000,2.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,0.992254,0.000000,0.887464,1.879718
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,1.000000,0.000000,1.000000,2.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,1.000000,1.000000,1.000000,3.000000
10,83,65,1,84,84,82,женщина,увидела,спальни,увидела,0.000002,0.999849,0.999849,1.999700
17,113,95,1,114,114,112,девушка,напугала,которых,напугала,0.182031,0.668873,0.668873,1.519777
23,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,0.367266,0.927165,0.927165,2.221596
32,145,142,0,146,146,-1,женщину,подкараулили,NaN,подкараулили,0.000000,0.513451,0.513451,1.026901


Если ещё уменьшить количество кандидатов:

In [46]:
alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 5)

In [57]:
db = reader.get_bundles().first()
candidates_df = alg.get_candidates(db)
db.pymorphy['normal_form'] = db.pymorphy['normal_form'].apply(lambda x: alg.pronoun_replacer[x] if x in alg.pronoun_replacer else x)
extended_df = get_extended_df(db, alg.get_pronoun_parent(db, candidates_df))

max_indices = extended_df.groupby('pronoun_word_id')['sum'].idxmax().dropna().astype(int)
best_in_group = extended_df.loc[max_indices]
best_in_group

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,1.000000,0.000000,1.000000,2.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,1.000000,1.000000,2.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,1.000000,1.000000,2.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,0.992254,0.000000,0.887464,1.879718
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,1.000000,0.000000,1.000000,2.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,1.000000,1.000000,1.000000,3.000000
10,83,65,1,84,84,82,женщина,увидела,спальни,увидела,0.000002,0.999849,0.999849,1.999700
15,113,95,1,114,114,112,девушка,напугала,которых,напугала,0.335821,0.746716,0.746716,1.829253
19,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,0.516609,0.996253,0.996253,2.509114
26,145,142,0,146,146,-1,женщину,подкараулили,NaN,подкараулили,0.000000,0.534172,0.534172,1.068345


Иногда криво определяются родители, может, брать произведения с соседями x2?

In [94]:
extended_df['left_prod_e'] = extended_df['left_prod_e'] * 2
extended_df['right_prod_e'] = extended_df['right_prod_e'] * 2
extended_df['sum'] = extended_df[['left_prod_e', 'right_prod_e', 'parent_prod_e']].sum(axis=1)
max_indices = extended_df.groupby('pronoun_word_id')['sum'].idxmax().dropna().astype(int)
best_in_group = extended_df.loc[max_indices]
best_in_group

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,8.000000,0.000000,1.000000,9.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,8.000000,1.000000,9.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,8.000000,1.000000,9.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,7.938034,0.000000,0.887464,8.825498
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,8.000000,0.000000,1.000000,9.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,8.000000,8.000000,1.000000,17.000000
10,83,65,1,84,84,82,женщина,увидела,спальни,увидела,0.000014,7.998792,0.999849,8.998656
15,113,95,1,114,114,112,девушка,напугала,которых,напугала,2.686569,5.973727,0.746716,9.407012
19,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,4.132870,7.970020,0.996253,13.099143
26,145,142,0,146,146,-1,женщину,подкараулили,NaN,подкараулили,0.000000,4.273380,0.534172,4.807552


In [74]:
l = extended_df[extended_df.pronoun_word_id == 174]
l

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
32,174,95,4,175,175,173,девушке,показалось,сначала,показалось,0.301898,2.937042,0.734260,3.973200
33,174,106,3,175,175,173,стене,показалось,сначала,показалось,0.184995,0.261641,0.065410,0.512047
34,174,118,2,175,175,173,голове,показалось,сначала,показалось,3.421241,0.473920,0.118480,4.013641
35,174,124,1,175,175,173,тревоге,показалось,сначала,показалось,0.001085,0.000029,0.000007,0.001121
36,174,142,0,175,175,173,женщине,показалось,сначала,показалось,0.090782,0.327367,0.081842,0.499991


In [131]:
def filter_max_sums(df):
    first_max_indices = df.groupby('pronoun_word_id')['sum'].idxmax()
    first_maxes = df.loc[first_max_indices][['pronoun_word_id', 'sum']]
    without_maxes = df.drop(first_max_indices.to_list())
    second_maxes = without_maxes.loc[without_maxes.groupby('pronoun_word_id')['sum'].idxmax()][['pronoun_word_id', 'sum']]
    cmp = first_maxes.merge(second_maxes, on='pronoun_word_id', how='left').fillna(0)
    cmp['diff'] = cmp['sum_x'] - cmp['sum_y']
    cmp['std'] = extended_df.groupby('pronoun_word_id')['sum'].std().fillna(0).reset_index(drop=True)
    good_diff = cmp[cmp['diff'] > cmp['std']]['pronoun_word_id'].to_list()
    return first_max_indices[good_diff].to_list()
    
extended_df.loc[filter_max_sums(extended_df)]

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,8.000000,0.000000,1.000000,9.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,8.000000,1.000000,9.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,8.000000,1.000000,9.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,7.938034,0.000000,0.887464,8.825498
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,8.000000,0.000000,1.000000,9.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,8.000000,8.000000,1.000000,17.000000
15,113,95,1,114,114,112,девушка,напугала,которых,напугала,2.686569,5.973727,0.746716,9.407012
19,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,4.132870,7.970020,0.996253,13.099143
27,160,95,4,163,161,-1,девушка,даже,NaN,успела,0.000000,4.922525,0.870104,5.792629
37,187,95,4,188,188,186,девушку,глазах,в,глазах,2.973235,3.470133,0.433767,6.877135


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Ниже использована последняя версия класса AntecedentCandidatesAlgorithm, можно запускать код

In [1]:
from tg.grammar_ru.ml.corpus import CorpusReader
from tg.grammar_ru.common import Loc
from pathlib import Path

base_path = Loc.corpus_path/'example.base.zip'
reader = CorpusReader(Path(base_path))
db = reader.get_bundles().first() 

In [2]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg5 = AntecedentCandidatesAlgorithm(reader.get_bundles(), 5)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


In [3]:
import pandas as pd

def view_results(db, result_df):
    return AntecedentCandidatesAlgorithm.get_context_words(db, result_df)[['pronoun_word_id', 'candidate_word_id', 'candidate_distance', 'left_word_1', 'candidate', 'right_word_1', 'parent_word', 'score']]

def get_diff_between_results(df1, df2):
    pairs1 = df1[['pronoun_word_id', 'candidate_word_id']]
    pairs2 = df2[['pronoun_word_id', 'candidate_word_id']]
    merged = pairs1.merge(pairs2, how = 'outer', indicator=True)
    left_indexes = merged[merged['_merge'] == 'left_only']['pronoun_word_id'].to_list()
    right_indexes = merged[merged['_merge'] == 'right_only']['pronoun_word_id'].to_list()
    return df1[df1['pronoun_word_id'].isin(left_indexes)], df2[df2['pronoun_word_id'].isin(right_indexes)]

#default
res = alg5.run_full(db, inflect=True, neighbour_count=1, neighbour_coeff=1, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
def_res = view_results(db, res) 
def_res

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,2.000000
1,18,0,0,NaN,женщина,гналась,гналась,2.000000
2,30,0,0,NaN,женщине,оставался,оставался,2.000000
3,48,6,2,от,сна,NaN,оторваться,1.879718
4,53,0,0,подвели,женщину,NaN,подвели,2.000000
5,56,0,0,и,женщина,упала,упала,3.000000
6,113,95,2,которых,девушка,напугала,напугала,1.832698
7,122,95,3,и,девушка,чувствовала,чувствовала,2.472011
8,145,142,0,NaN,женщину,подкараулили,подкараулили,1.984920
9,160,142,0,NaN,женщина,даже,успела,1.794923


In [4]:
res = alg5.run_full(db, inflect=True, neighbour_coeff=1, use_diff_between_second_filter=False,  use_diff_between_mean_filter=True)
mean_filter_res = view_results(db, res)
mean_filter_res

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,2.000000
1,18,0,0,NaN,женщина,гналась,гналась,2.000000
2,30,0,0,NaN,женщине,оставался,оставался,2.000000
3,48,6,2,от,сна,NaN,оторваться,1.879718
4,53,0,0,подвели,женщину,NaN,подвели,2.000000
5,56,0,0,и,женщина,упала,упала,3.000000
6,113,95,2,которых,девушка,напугала,напугала,1.832698
7,122,95,3,и,девушка,чувствовала,чувствовала,2.472011
8,145,142,0,NaN,женщину,подкараулили,подкараулили,1.984920
9,160,142,0,NaN,женщина,даже,успела,1.794923


In [5]:
left, right = get_diff_between_results(def_res, mean_filter_res)
print(len(left))
right

0


,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
11,187,124,1,в,тревогу,глазах,глазах,1.071350
20,432,427,0,из,женщины,победительницей,выйти,1.889009
26,583,579,0,остановит,мерзавца,сама,остановит,1.157367
27,594,556,1,для,женщины,преградой,станут,1.801942
28,599,535,4,NaN,дом,и,могла,1.150486
29,621,566,2,когда,постели,угодно,знал,1.890681
39,874,848,2,в,пульсации,NaN,таившаяся,0.980214


Вывод: лучше отсеивать по разнице со вторым максимумом

In [6]:
res = alg5.run_full(db, inflect=True, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
coef_2_res = view_results(db, res)
coef_2_res

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,3.000000
1,18,0,0,NaN,женщина,гналась,гналась,3.000000
2,30,0,0,NaN,женщине,оставался,оставался,3.000000
3,48,6,2,от,сна,NaN,оторваться,2.871972
4,53,0,0,подвели,женщину,NaN,подвели,3.000000
5,56,0,0,и,женщина,упала,упала,5.000000
6,113,95,2,которых,девушка,напугала,напугала,2.916493
7,122,95,3,и,девушка,чувствовала,чувствовала,3.948728
8,145,142,0,NaN,женщину,подкараулили,подкараулили,2.977380
9,160,142,0,NaN,женщина,даже,успела,2.652213


In [7]:
left, right = get_diff_between_results(def_res, coef_2_res)
print(len(right))
left

0


,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
14,335,333,0,NaN,женщину,хотел,убить,1.469332
20,489,486,0,соединяющую,ниточку,с,соединяющую,2.014223


In [8]:
db.src.loc[331:335]

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id
331,331,24,10,4,1,",",punct,1,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
332,332,24,11,4,1,сонных,ru,6,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
333,333,24,12,4,0,Чисовицах,ru,9,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
334,334,24,13,4,1,.,punct,1,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
335,335,25,0,4,1,Ее,ru,2,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip


Вывод: удвоение веса произведений с соседями может отрезать ошибки

In [9]:
res = alg5.run_full(db, inflect=True, neighbour_coeff=4, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
coef_4_res = view_results(db, res)
coef_4_res

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,5.000000
1,18,0,0,NaN,женщина,гналась,гналась,5.000000
2,30,0,0,NaN,женщине,оставался,оставался,5.000000
3,48,6,2,от,сна,NaN,оторваться,4.856481
4,53,0,0,подвели,женщину,NaN,подвели,5.000000
5,56,0,0,и,женщина,упала,упала,9.000000
6,113,95,2,которых,девушка,напугала,напугала,5.084082
7,122,95,3,и,девушка,чувствовала,чувствовала,6.902161
8,145,142,0,NaN,женщину,подкараулили,подкараулили,4.962299
9,160,142,0,NaN,женщина,даже,успела,4.366792


In [10]:
left, right = get_diff_between_results(coef_4_res, coef_2_res)
print(len(right), len(left))

0 0


Не увлекаться, небольшого коэффициента достаточно.

In [11]:
res = alg5.run_full(db, inflect=True, neighbour_count=2, neighbour_coeff=1, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
neighb_2 = view_results(db, res) 
neighb_2

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,4.000000
1,18,0,0,NaN,женщина,гналась,гналась,4.000000
2,30,0,0,NaN,женщине,оставался,оставался,4.000000
3,48,6,2,от,сна,NaN,оторваться,3.730963
4,53,0,0,подвели,женщину,NaN,подвели,4.000000
5,56,0,0,и,женщина,упала,упала,4.000000
6,83,65,1,спальни,женщина,увидела,увидела,3.249932
7,113,95,2,которых,девушка,напугала,напугала,2.772502
8,122,95,3,и,девушка,чувствовала,чувствовала,3.104682
9,145,142,0,NaN,женщину,подкараулили,подкараулили,2.716782


In [12]:
left, right = get_diff_between_results(def_res, neighb_2)
left

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
16,365,272,4,NaN,голова,была,звездой,1.549724
23,581,541,2,NaN,тьма,остановит,остановит,1.518738
26,691,669,4,где,тишина,NaN,где,1.397047
27,693,669,4,NaN,тишина,прошла,прошла,1.318750
31,833,803,3,в,боль,глазах,глазах,2.286138
32,858,829,4,билось,тишину,сердце,сердце,1.572379
34,917,907,1,уготовила,жертве,роли,уготовила,2.124729
35,921,896,4,NaN,женщина,кубарем,кубарем,0.972968


In [13]:
right

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
6,83,65,1,спальни,женщина,увидела,увидела,3.249932
19,432,427,0,из,женщины,победительницей,выйти,2.829493
24,594,556,1,для,женщины,преградой,станут,2.660689
30,874,848,2,в,пульсации,NaN,таившаяся,2.620805


Убрано 7 ошибок и 1 правильный кандидат, добавлено 2 ошибки и 2 правильных кандидата

In [14]:
res = alg5.run_full(db, inflect=True, neighbour_count=2, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
neighb_2_coef_2 = view_results(db, res) 
neighb_2_coef_2

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,7.000000
1,18,0,0,NaN,женщина,гналась,гналась,7.000000
2,30,0,0,NaN,женщине,оставался,оставался,7.000000
3,48,6,2,от,сна,NaN,оторваться,6.574462
4,53,0,0,подвели,женщину,NaN,подвели,7.000000
5,56,0,0,и,женщина,упала,упала,7.000000
6,83,65,1,спальни,женщина,увидела,увидела,5.500015
7,113,95,2,которых,девушка,напугала,напугала,4.796100
8,122,95,3,и,девушка,чувствовала,чувствовала,5.214069
9,145,142,0,NaN,женщину,подкараулили,подкараулили,4.441105


In [15]:
left, right = get_diff_between_results(def_res, neighb_2_coef_2)
left

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
16,365,272,4,NaN,голова,была,звездой,1.549724
23,581,541,2,NaN,тьма,остановит,остановит,1.518738
26,691,669,4,где,тишина,NaN,где,1.397047
27,693,669,4,NaN,тишина,прошла,прошла,1.318750
28,755,749,0,NaN,женщина,приготовилась,приготовилась,1.988473
31,833,803,3,в,боль,глазах,глазах,2.286138
32,858,829,4,билось,тишину,сердце,сердце,1.572379
34,917,907,1,уготовила,жертве,роли,уготовила,2.124729
35,921,896,4,NaN,женщина,кубарем,кубарем,0.972968


In [16]:
right

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
6,83,65,1,спальни,женщина,увидела,увидела,5.500015
19,432,427,0,из,женщины,победительницей,выйти,4.718502
24,583,536,2,остановит,мужчину,сама,остановит,3.507455
25,594,556,1,для,женщины,преградой,станут,4.462632
28,693,671,3,NaN,женщина,прошла,прошла,4.272386
31,874,848,2,в,пульсации,NaN,таившаяся,4.261482


In [17]:
left, right = get_diff_between_results(neighb_2, neighb_2_coef_2)
right

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
24,583,536,2,остановит,мужчину,сама,остановит,3.507455
28,693,671,3,NaN,женщина,прошла,прошла,4.272386


Увеличение окна соседей улучшает результат

In [18]:
res = alg5.run_full(db, inflect=True, neighbour_count=4, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
neighb_4_coef_2 = view_results(db, res) 
neighb_4_coef_2

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,11.000000
1,18,0,0,NaN,женщина,гналась,гналась,11.000000
2,30,0,0,NaN,женщине,оставался,оставался,13.000000
3,48,6,2,от,сна,NaN,оторваться,11.262703
4,53,0,0,подвели,женщину,NaN,подвели,13.000000
5,56,0,0,и,женщина,упала,упала,13.000000
6,83,65,1,спальни,женщина,увидела,увидела,7.424463
7,145,142,0,NaN,женщину,подкараулили,подкараулили,6.628748
8,160,142,0,NaN,женщина,даже,успела,7.922376
9,174,118,2,сначала,голове,показалось,показалось,5.965538


In [19]:
left, right = get_diff_between_results(neighb_2_coef_2, neighb_4_coef_2)
left

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
7,113,95,2,которых,девушка,напугала,напугала,4.796100
8,122,95,3,и,девушка,чувствовала,чувствовала,5.214069
16,351,303,4,NaN,счёт,имел,имел,3.555585
24,583,536,2,остановит,мужчину,сама,остановит,3.507455
27,666,556,4,вокруг,женщины,была,NaN,4.667187
28,693,671,3,NaN,женщина,прошла,прошла,4.272386
30,799,782,1,NaN,женщина,ожидала,ожидала,5.150627
31,874,848,2,в,пульсации,NaN,таившаяся,4.261482


In [20]:
right

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
12,281,272,1,не,голову,страх,страх,8.799526
21,515,504,0,перед,женщине,стоял,стоял,5.390777
23,581,556,1,NaN,женщина,остановит,остановит,7.343985
26,755,749,0,NaN,женщина,приготовилась,приготовилась,7.648712
28,790,787,0,страсть,силу,были,страсть,5.502509
29,833,831,0,в,темноту,глазах,глазах,7.630587
31,917,896,2,уготовила,женщине,роли,уготовила,10.527887
32,921,896,4,NaN,женщина,кубарем,кубарем,6.515668


Увеличение окна до 4 не сильно помогает.

In [21]:
res = alg5.run_full(db, inflect=True, neighbour_count=3, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
neighb_3_coef_2 = view_results(db, res) 
neighb_3_coef_2

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,9.000000
1,18,0,0,NaN,женщина,гналась,гналась,11.000000
2,30,0,0,NaN,женщине,оставался,оставался,9.000000
3,48,6,2,от,сна,NaN,оторваться,9.293710
4,53,0,0,подвели,женщину,NaN,подвели,11.000000
5,56,0,0,и,женщина,упала,упала,9.000000
6,113,95,2,которых,девушка,напугала,напугала,5.006380
7,122,95,3,и,девушка,чувствовала,чувствовала,5.214087
8,145,142,0,NaN,женщину,подкараулили,подкараулили,6.563811
9,160,142,0,NaN,женщина,даже,успела,6.445712


In [22]:
left, right = get_diff_between_results(neighb_2_coef_2, neighb_3_coef_2)
left

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
6,83,65,1,спальни,женщина,увидела,увидела,5.500015
16,351,303,4,NaN,счёт,имел,имел,3.555585
27,666,556,4,вокруг,женщины,была,NaN,4.667187
28,693,671,3,NaN,женщина,прошла,прошла,4.272386
30,799,782,1,NaN,женщина,ожидала,ожидала,5.150627
31,874,848,2,в,пульсации,NaN,таившаяся,4.261482


In [23]:
right

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
13,281,272,1,не,голову,страх,страх,5.614914
22,515,504,0,перед,женщине,стоял,стоял,4.724446
24,581,556,1,NaN,женщина,остановит,остановит,5.611024
27,621,566,2,когда,постели,угодно,знал,5.133458
30,790,787,0,страсть,силу,были,страсть,4.631253
32,917,896,2,уготовила,женщине,роли,уготовила,6.946753
33,921,896,4,NaN,женщина,кубарем,кубарем,4.519860


Вариант `neighb_2_coef_2` был самым удачным, посмотрим на него ещё раз:

In [24]:
neighb_2_coef_2

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,7.000000
1,18,0,0,NaN,женщина,гналась,гналась,7.000000
2,30,0,0,NaN,женщине,оставался,оставался,7.000000
3,48,6,2,от,сна,NaN,оторваться,6.574462
4,53,0,0,подвели,женщину,NaN,подвели,7.000000
5,56,0,0,и,женщина,упала,упала,7.000000
6,83,65,1,спальни,женщина,увидела,увидела,5.500015
7,113,95,2,которых,девушка,напугала,напугала,4.796100
8,122,95,3,и,девушка,чувствовала,чувствовала,5.214069
9,145,142,0,NaN,женщину,подкараулили,подкараулили,4.441105


7/33 ошибок, попробуем менять допустимое кол-во кандидатов:

In [25]:
alg6 = AntecedentCandidatesAlgorithm(reader.get_bundles(), 6)
res = alg6.run_full(db, inflect=True, neighbour_count=2, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
view_results(db, res)

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,7.000000
1,18,0,0,NaN,женщина,гналась,гналась,7.000000
2,30,0,0,NaN,женщине,оставался,оставался,7.000000
3,48,6,2,от,сна,NaN,оторваться,6.574462
4,53,0,0,подвели,женщину,NaN,подвели,7.000000
5,56,0,0,и,женщина,упала,упала,7.000000
6,83,65,1,спальни,женщина,увидела,увидела,5.500015
7,113,95,2,которых,девушка,напугала,напугала,4.606171
8,122,95,3,и,девушка,чувствовала,чувствовала,5.135611
9,224,208,0,NaN,женщина,мысленно,обратилась,4.473188


In [26]:
alg4 = AntecedentCandidatesAlgorithm(reader.get_bundles(), 4)
res = alg4.run_full(db, inflect=True, neighbour_count=2, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
view_results(db, res)

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,7.000000
1,18,0,0,NaN,женщина,гналась,гналась,7.000000
2,30,0,0,NaN,женщине,оставался,оставался,7.000000
3,48,6,2,от,сна,NaN,оторваться,6.574462
4,53,0,0,подвели,женщину,NaN,подвели,7.000000
5,56,0,0,и,женщина,упала,упала,7.000000
6,83,65,1,спальни,женщина,увидела,увидела,5.500015
7,113,95,2,которых,девушка,напугала,напугала,5.521651
8,122,95,3,и,девушка,чувствовала,чувствовала,5.250813
9,145,142,0,NaN,женщину,подкараулили,подкараулили,4.801946


In [27]:
alg3 = AntecedentCandidatesAlgorithm(reader.get_bundles(), 3)
res = alg3.run_full(db, inflect=True, neighbour_count=2, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
view_results(db, res)

,pronoun_word_id,candidate_word_id,candidate_distance,left_word_1,candidate,right_word_1,parent_word,score
0,10,0,0,преследовал,женщину,NaN,преследовал,7.000000
1,18,0,0,NaN,женщина,гналась,гналась,7.000000
2,30,0,0,NaN,женщине,оставался,оставался,7.000000
3,48,6,2,от,сна,NaN,оторваться,6.577227
4,53,0,0,подвели,женщину,NaN,подвели,7.000000
5,56,0,0,и,женщина,упала,упала,7.000000
6,83,65,1,спальни,женщина,увидела,увидела,5.500015
7,113,95,2,которых,девушка,напугала,напугала,5.984794
8,145,142,0,NaN,женщину,подкараулили,подкараулили,4.837285
9,160,142,0,NaN,женщина,даже,успела,4.818550


In [28]:
db.src.loc[587:610]

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id
587,587,43,1,7,1,колдун,ru,6,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
588,588,43,2,7,0,думал,ru,5,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
589,589,43,3,7,1,",",punct,1,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
590,590,43,4,7,1,что,ru,3,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
591,591,43,5,7,1,двери,ru,5,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
592,592,43,6,7,1,станут,ru,6,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
593,593,43,7,7,1,для,ru,3,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
594,594,43,8,7,1,нее,ru,3,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
595,595,43,9,7,0,преградой,ru,9,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip
596,596,43,10,7,1,?,punct,1,cecdfcc5-63d7-4765-bce6-1368afc1041d,example.base.zip


На этом примере отступ в 5 кандидатов оказался лучшим. Попробуем другой отрывок текста.

In [29]:
new_db = list(reader.get_bundles())[1]
new_db.src

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id
10926,10926,10926,0,10926,1,Полоска,ru,7,0,0,0,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
10927,10927,10926,1,10926,1,утреннего,ru,9,1,0,0,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
10928,10928,10926,2,10926,1,света,ru,5,2,0,0,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
10929,10929,10926,3,10926,1,из,ru,2,3,0,0,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
10930,10930,10926,4,10926,1,щели,ru,4,4,0,0,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15901,15901,11253,14,11055,1,бы,ru,2,4975,327,129,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
15902,15902,11253,15,11055,1,побрезговали,ru,12,4976,327,129,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
15903,15903,11253,16,11055,1,этим,ru,4,4977,327,129,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip
15904,15904,11253,17,11055,0,неумехой,ru,8,4978,327,129,False,d1d0afe5-ab98-4d85-a9b4-132f5d76ecd2,example.base.zip


In [30]:
res = alg5.run_full(new_db, inflect=True, neighbour_count=2, neighbour_coeff=2, use_diff_between_second_filter=True,  use_diff_between_mean_filter=False)
view_results(new_db, res)

AttributeError: 'NoneType' object has no attribute 'word'

Ладно, надо где-то чинить код...